In [1]:
import cadquery as cq
import numpy as np 
from math import pi, sin, cos, sqrt

# Debugging code

In [17]:
import cadquery as cq


def tidy_repr(obj):
    """Shortens a default repr string"""
    return repr(obj).split(".")[-1].rstrip(">")


def _ctx_str(self):
    return (
        tidy_repr(self)
        + ":\n"
        + f"    pendingWires: {self.pendingWires}\n"
        + f"    pendingEdges: {self.pendingEdges}\n"
        + f"    tags: {self.tags}"
    )


cq.cq.CQContext.__str__ = _ctx_str


def _plane_str(self):
    return (
        tidy_repr(self)
        + ":\n"
        + f"    origin: {self.origin.toTuple()}\n"
        + f"    z direction: {self.zDir.toTuple()}"
    )


cq.occ_impl.geom.Plane.__str__ = _plane_str


def _wp_str(self):
    out = tidy_repr(self) + ":\n"
    out += f"  parent: {tidy_repr(self.parent)}\n" if self.parent else "  no parent\n"
    out += f"  plane: {self.plane}\n"
    out += f"  objects: {self.objects}\n"
    out += f"  modelling context: {self.ctx}"
    return out


cq.Workplane.__str__ = _wp_str

# Circle from parametricCurve

In [3]:

def circle(t, r):
    #if t == 1:
    #    t = 0
    x = r * cos(t)
    #y = sqrt(r**2 - x**2)
    y = r * sin(t)
    #(x−a)2 + (y−b)2 = r2.
    if t == 0 or t == 2 * pi:
        print(f"t: {t}; x: {x}; y: {y}")
    return(x, y, 0)

r = 10
result = (
    cq.Workplane("XY")
    .parametricCurve(lambda t: circle(t, r), N=10, start=0, stop=2*pi, makeWire=True)
#    .close()
    .circle(r/2)
    .extrude(1)
)
display(result)

t: 0.0; x: 10.0; y: 0.0
t: 6.283185307179586; x: 10.0; y: -2.4492935982947065e-15


# Circle from using previouse method but low poly

In [4]:


def circle(t, r):
    #if t == 1:
    #    t = 0
    x = r * cos(t)
    #y = sqrt(r**2 - x**2)
    y = r * sin(t)
    #(x−a)2 + (y−b)2 = r2.
    if t == 0 or t == 2 * pi:
        print(f"t: {t}; x: {x}; y: {y}")
    return(round(x, 5), round(y, 5))

r = 10

num_steps = 10
max = 2 * pi

points = [circle(i, r) for i in np.linspace(0, max, num_steps)]
print(points)

result = (
    cq.Workplane("XY")
    .polyline(points)
    .close()
    .extrude(1)
)
display(result)

t: 0.0; x: 10.0; y: 0.0
t: 6.283185307179586; x: 10.0; y: -2.4492935982947065e-15
[(10.0, 0.0), (7.66044, 6.42788), (1.73648, 9.84808), (-5.0, 8.66025), (-9.39693, 3.4202), (-9.39693, -3.4202), (-5.0, -8.66025), (1.73648, -9.84808), (7.66044, -6.42788), (10.0, -0.0)]


# Create a sphere

In [5]:
def sphere(u, v, r):
    x = r * sin(u) * cos(v)
    y = r * sin(u) * sin(v)
    z = r * cos(u)
    return (x, y, z)

r = 10
result = (
    cq.Workplane("XY")
    .parametricSurface(lambda u, v: sphere(u, v*2, r), N=100, start=0, stop=pi).val()
)
display(result)

# Create a solid sphere

In [6]:
def sphere(u, v, r):
    x = r * sin(u) * cos(v)
    y = r * sin(u) * sin(v)
    z = r * cos(u)
    return (x, y, z)

r = 10
face = (
    cq.Workplane("XY")
    .parametricSurface(lambda u, v: sphere(u, v*2, r), N=10, start=0, stop=pi)
)
edges = [
    face.edges(">X").val(),
    face.edges("<X").val(),
]
print(edges)
end_wires = [cq.Wire.assembleEdges([e]) for e in edges]
end_faces = [cq.Face.makeFromWires(w, []) for w in end_wires]
faces = [face.val()] + end_faces
shell = cq.Shell.makeShell(faces).fix()

print(f"{shell.isValid()=}") # True
print(f"{shell.Closed()=}") # False

solid = cq.Solid.makeSolid(shell)
display(solid)

[<cadquery.occ_impl.shapes.Edge object at 0x4084a4b290>, <cadquery.occ_impl.shapes.Edge object at 0x407f8aad90>]
shell.isValid()=False
shell.Closed()=False


In [7]:
def sphere(u, v, r):
    x = r * sin(u) * cos(v)
    y = r * sin(u) * sin(v)
    z = r * cos(u)
    return (x, y, z)

r = 10
sphere = (
    cq.Workplane("XY")
    .parametricSurface(lambda u, v: sphere(u, v*2, r), N=10, start=0, stop=pi)
)
r=10
result = (
    cq.Workplane().box(2*r,2*r,2*r)
    .split(cq.Workplane().add(sphere))
    .solids(">X")
)
#print(result.solids())
display(result)

# Acustic pattern

In [8]:
from math import *
from cadquery import *


f = Workplane().parametricSurface(
    lambda u, v: (u, v, 0.5*cos(pi * u) * cos(pi * v)),N=40, tol=1e-6, start=-2.1, stop=2.1
)
f


In [9]:
from math import *
from cadquery import *


f = Workplane().parametricSurface(
    lambda u, v: (u, v, 0.5*cos(pi * u) * cos(pi * v)),N=40, tol=1e-6, start=-2.1, stop=2.1
)

res = (
    cq.Workplane().box(4,4,2)
    .split(cq.Workplane().add(f))
    .solids("<Z")
    .edges('|Z').fillet(.5)
)

display(res)

# Experimental

In [10]:
def curve(x, a, b=0, c=0):
    y = (a * x**2) + (b * x) + c
    return (x, y)

a = 2
result = (
    cq.Workplane("XY")
    .parametricCurve(lambda u: curve(u, a), N=40, start=-1, stop=1)
    .twistExtrude(15, 90)
)

display(result)

In [11]:
def curve(x, a, b=0, c=0):
    y = (a * x**2) + (b * x) + c
    return (x, y)

a = 2
result = (
    cq.Workplane("XY")
    .parametricCurve(lambda u: curve(u, a), N=40, start=-1, stop=1, makeWire=False)
    .close()
    .twistExtrude(15, 90)
)

display(result)

In [12]:
def parabola(x, a, b=0, c=0):
    y = (a * x**2) + (b * x) + c
    return (x, y)

a = 2
face = (
    cq.Workplane("XY")
    .parametricCurve(lambda u: parabola(u, a), N=40, start=-1, stop=0)#.val()
    #.close()
    #.Workplane()
    .revolve(360, (0, 0, 0), (0, -1, 0))
)


result = (
    cq.Workplane().box(1,1,1)
    .split(cq.Workplane().add(face))
    #.solids("<Z[1]")
)


display(face)

In [13]:
def parabola(x, a, b=0, c=0):
    y = (a * x**2) + (b * x) + c
    return (x, y)

In [14]:
curve = (
    cq.Workplane("XY")
    .parametricCurve(lambda u: parabola(u, a), N=40, start=-1, stop=1, makeWire=False)
    .close()
    .revolve(360, (1.5, 0, 0), (1.5, -5, 0))
)
display(curve)


In [15]:
import cadquery as cq

result = cq.Workplane().radiusArc((0, 2, 0), 1).close()
result = result.revolve(360, (0, 0, 0), (0, 1, 0))
result

In [16]:
def parabola(x, a, b=0, c=0):
    y = (a * x**2) + (b * x) + c
    return (x, y)
    
a = 1.5
stop = 1
final_height = parabola(stop, a)[1]
result = (
    cq.Workplane("XY")
    .parametricCurve(lambda u: parabola(u, a), N=40, start=0, stop=1, makeWire=False)
    .lineTo(0, final_height)
    .close()
    .revolve(360, (0, 0, 0), (0, -5, 0))
)
display(result)